In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/audio-samples/Audio_Samples'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# This is for Ignoreing unwanted WARNINGS

import warnings
warnings.filterwarnings('ignore')
print("DONE!")

DONE!


In [3]:
!pip install -q openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 26.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.7 MB/

In [4]:
import whisper
model = whisper.load_model("base", device="cpu")
print("DONE!")

# result = model.transcribe(audio_path)
# text = result["text"]
# print("text: ",text)

100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 258MiB/s]


DONE!


**Load Whisper Model (Speech → Text)**

In [5]:
import re

def clean_text(text):
    if not isinstance(text, str):
        return ""

    # 1. Convert text to Lowercase
    text = text.lower()

    # 2. Remove fillers common in speech
    fillers = [
        r"\buh\b", r"\bum\b", r"\bhmm\b", r"\bah\b",
        r"\ber\b", r"\beh\b"
    ]
    for f in fillers:
        text = re.sub(f, "", text)

    # 3. Remove repeated words from text
    text = re.sub(r'\b(\w+)( \1\b)+', r'\1', text)

    # 4. Remove unwanted characters from text
    text = re.sub(r"[^a-z0-9\s\.\,\?\!]", "", text)

    # 5. Fix extra spaces
    text = re.sub(r"\s+", " ", text).strip()
    
    return text

# text = "I I think this is is a very very good example. It show grammar mistake."
# print(clean_text(text))
print("Done")

Done


**Text Cleaning (Speech-Specific)**
Remove fillers, repetitions, and noise without correcting grammar.

In [6]:
import nltk
# import re 
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger_eng')

print("DONE!")

DONE!


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


In [7]:
def extract_grammar_features(text):
    if not isinstance(text, str) or not text.strip():
        return {
            "error_ratio": 1.0,
            "avg_sentence_length": 0.0,
            "pos_consistency": 0.0,
            "repetition_ratio": 1.0
        }

    sentences = nltk.sent_tokenize(text)
    words = nltk.word_tokenize(text)

    if len(words) == 0:
        return {
            "error_ratio": 1.0,
            "avg_sentence_length": 0.0,
            "pos_consistency": 0.0,
            "repetition_ratio": 1.0
        }

    # Sentence length
    avg_sentence_length = len(words) / max(len(sentences), 1)

    # Repetition
    from collections import Counter
    word_counts = Counter(words)
    repetition_ratio = sum(v for v in word_counts.values() if v > 1) / len(words)

    # POS tagging (stable)
    pos_tags = nltk.pos_tag(words, lang='eng')
    pos_counts = Counter(tag for _, tag in pos_tags)
    pos_consistency = max(pos_counts.values()) / len(pos_tags)

    # Error proxy
    grammar_errors = 0
    for i in range(len(pos_tags) - 1):
        t1 = pos_tags[i][1]
        t2 = pos_tags[i + 1][1]

        if t1 == t2 and t1 in ["NN", "VB", "JJ"]:
            grammar_errors += 1
        if t1 == "DT" and t2.startswith("VB"):
            grammar_errors += 1

    error_ratio = grammar_errors / len(words)

    return {
        "error_ratio": round(error_ratio, 4),
        "avg_sentence_length": round(avg_sentence_length, 2),
        "pos_consistency": round(pos_consistency, 4),
        "repetition_ratio": round(repetition_ratio, 4)
    }
# text = "I I think this is is a very very good example. It show grammar mistake."
# print(extract_grammar_features(text))
print("DONE!")

DONE!


**Grammar Feature Extraction**

In [8]:
def grammar_score(features):
    error_ratio = features["error_ratio"]
    repetition_ratio = features["repetition_ratio"]
    avg_sentence_length = features["avg_sentence_length"]
    pos_consistency = features["pos_consistency"]


    # Lower error 
    error_score = max(0, 1 - error_ratio * 2)

    # Lower repetition
    repetition_score = max(0, 1 - repetition_ratio * 2)

    # Sentence length sweet spot (8–20)
    if avg_sentence_length < 8:
        length_score = avg_sentence_length / 8
    elif avg_sentence_length > 20:
        length_score = max(0, 1 - (avg_sentence_length - 20) / 20)
    else:
        length_score = 1.0

    # POS consistency sweet range
    if 0.3 <= pos_consistency <= 0.55:
        pos_score = 1.0
    else:
        pos_score = max(0, 1 - abs(pos_consistency - 0.425) * 2)

    # Weighted Sum for Final Grammar score
    final_score = (
        0.4 * error_score +
        0.25 * repetition_score +
        0.2 * length_score +
        0.15 * pos_score
    )

    # Scale to 0–10
    return round(final_score * 10, 2)


# print(grammar_score(features))
    
print("DONE!")

DONE!


**Rule-Based Grammar Scoring (0–10)**

In [9]:
#Base DIR of sample files
AUDIO_DIR = '/kaggle/input/grammar-voice-samples/Audio_Samples'

#Extracting Sample file path one by one and storeing
audio_files = [
    os.path.join(AUDIO_DIR, f)
    for f in os.listdir(AUDIO_DIR)
    if f.endswith(".mp3")
]
# print(audio_files)
print("DONE!")

DONE!


In [10]:
import pandas as pd

results = []

for audio_path in audio_files:
    # 1. Convert Text fromthe .mp3 voice samples by the WHISHPER
    
    #HOPE WHISHPER MODEL LOADED SUCCESSFULLY!
    result = model.transcribe(audio_path)
    raw_text = result["text"]

    # 2. Text Cleaning
    clean_text_data = clean_text(raw_text)

    # 3. Extract grammar features
    features = extract_grammar_features(clean_text_data)

    # 4. Getting Final Grammar score
    score = grammar_score(features)

    results.append({
        "audio_file": os.path.basename(audio_path),
        "transcription": clean_text_data,
        "error_ratio": features["error_ratio"],
        "avg_sentence_length": features["avg_sentence_length"],
        "pos_consistency": features["pos_consistency"],
        "repetition_ratio": features["repetition_ratio"],
        "grammar_score": score
    })

#Final Output
df_results = pd.DataFrame(results)
df_results


,audio_file,transcription,error_ratio,avg_sentence_length,pos_consistency,repetition_ratio,grammar_score
0,sample2.mp3,"she likes football, play 2.",0.0000,7.0,0.1429,0.0,8.90
1,sample3.mp3,the dog is barking loudly.,0.0000,6.0,0.1667,0.0,8.73
2,sample6.mp3,we have working being on this project.,0.0000,8.0,0.2500,0.0,9.48
3,sample1.mp3,is dog the barking loudly.,0.0000,6.0,0.1667,0.0,8.73
4,sample4.mp3,they were going to their park yesterday.,0.1250,8.0,0.2500,0.0,8.47
5,sample5.mp3,grandparents will visit my eye tomorrow.,0.1429,7.0,0.2857,0.0,8.19


**Process All Audio Files**

In [11]:
# Create output directory
output_dir = "/kaggle/working/output"
os.makedirs(output_dir, exist_ok=True)

# Save DataFrame to CSV
output_path = os.path.join(output_dir, "grammar_scores.csv")
df_results.to_csv(output_path, index=False)

print(f"Output saved at: {output_path}")


Output saved at: /kaggle/working/output/grammar_scores.csv


**Save Output to CSV**